<h2>The RSA crypto scheme, sometimes referred to as the
Rivest–Shamir–Adleman algorithm, is currently the
most widely used asymmetric cryptographic scheme</h2>
<h3> There are many applications for RSA, but in practice it
is most often used for:</h3><br />
1- encryption of small pieces of data, especially for key
transport <br />
2- digital signatures

<br /><h3>Here we are going to implement RSA with public and private keys up to 512 bits within limit of time for setup and decryption less than 3 seconds!<h3>

First let us import our neede libraries

In [1]:
import random
import numpy as np
from random import randrange, randint
import timeit

1- We need to implement Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers

In [2]:
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

Square and multiply to make expontiation faster than usual

In [3]:
def squareAndMultiply(base,exponent,modulus):
    #Converting the exponent to its binary form
    binaryExponent = []
    while exponent != 0:
        binaryExponent.append(exponent%2)
        exponent = exponent/2
    #Appllication of the square and multiply algorithm
    result = 1
    binaryExponent.reverse()
    for i in binaryExponent:
        if i == 0:
            result = (result*result) % modulus
        else:
            result = (result*result*base) % modulus

    return result

2- Euclid's extended algorithm for finding the multiplicative inverse of two numbers

In [4]:
def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi
    
    while e > 0:
        temp1 = temp_phi/e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2
        
        x = x2- temp1* x1
        y = d - temp1 * y1
        
        x2 = x1
        x1 = x
        d = y1
        y1 = y
    
    if temp_phi == 1:
        return d + phi

3- Tests to see if a number is prime. and to keep our time limit we need to use a trick to make our test faster so we implement Miller-rabin for primality test 

In [5]:
def is_prime(n, k=10):
    if n == 2:
        return True
    if not n & 1:
        return False

    def check(a, s, d, n):
        x = pow(a, d, n)
        if x == 1:
            return True
        for i in xrange(s - 1):
            if x == n - 1:
                return True
            x = pow(x, 2, n)
        return x == n - 1

    s = 0
    d = n - 1

    while d % 2 == 0:
        d >>= 1
        s += 1

    for i in xrange(k):
        a = randrange(2, n - 1)
        if not check(a, s, d, n):
            return False
    return True

4- here we are going to generat prime number according to the size entered bu the user

In [6]:
def generate_big_prime(n, p_in=0):
    found_prime = False
    while not found_prime:
        p = randint(pow(2, (n-1)), pow(2, n))
        if is_prime(p, 100) and p != p_in:
            return p

5- generate public and private key pair 

In [7]:
def generate_keypair(p, q, n):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    n = np.multiply(p , q)
    phi = np.multiply((p-1) , (q-1))
    e = random.randrange(1, phi)
    # Use Euclid's Algorithm to verify that e and phi(n) are comprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)
    # Use Extended Euclid's Algorithm to generate the private key
    d = multiplicative_inverse(e, phi)
    
    # Return public and private keypair
    # Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

6- here we are gonna finish we need Encryption and Decryption methods of our RSA

In [8]:
def encrypt(pk, plaintext):
    #Unpack the key into it's components
    key, n = pk
    cipher = [(squareAndMultiply(ord(char), key , n)) for char in plaintext]
    return cipher

In [9]:
def decrypt(pk, ciphertext):
    #Unpack the key into its components
    key, n = pk
    #Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr(squareAndMultiply(char , key , n)) for char in ciphertext]
    #Return the array of bytes as a string
    return ''.join(plain)

Finally our main method to call previous methods 

In [10]:
print "RSA Encrypter/ Decrypter"

RSA Encrypter/ Decrypter


In [11]:
p_size =int(raw_input("Enter size of P in bits : "))
q_size =int(raw_input("Enter size of q in bits : "))

Enter size of P in bits : 512
Enter size of q in bits : 512


In [12]:
setup_start = timeit.default_timer()
p = generate_big_prime(p_size)
q = generate_big_prime(q_size, p)

In [13]:
print('Value of P : ' + str(p))
print('Value of q : ' + str(q))

Value of P : 12696127601781339801856395950318397170597584398231246500265647455796978896809074929051461759357736489903053171427168235738151558854156946572297470007366599
Value of q : 12234530517596663832847179047145218648087097985514495581814091758232723659976704784474585216600166846973067867637033623139298943604802661823159592432796721


In [14]:
print "Generating your public/private keypairs now . . ."
public, private = generate_keypair(p, q, p_size)
print "Your public key is ", public ," and your private key is ", private

Generating your public/private keypairs now . . .
Your public key is  (65120766826150720824463739424754782284639417151760125041534888110709496909347216488683996762150072124585160960283905236804377908578967191024643684234067715337717881938459281608153251517827163210181855479225324822312654255039045719140931030426337608911374033172480813801834339216441771909365298972278085410571L, 155331160599295145524154527022186629413620703970199690003660283986364067501366992517713877630635372589859083873595194617473517367042043556154841408323091976069376596953768829043722315317989549565749906548766634158352464847040946713893097074169621752390132486327615865697714115763693287478600083051717892121879L)  and your private key is  (222030375980028671811862881649983524225936740214930498029093297814457023401757014574777899109847082291572893367728083139867142943687490350869082794892210123826920905610888683573126691868305413524295120725969080548200071126776892290562781080192388539520873145830214835403795990982

In [15]:
setup_end = timeit.default_timer()
print("setup time is : " + str((setup_end - setup_start)))

setup time is : 1.18098497391


In [16]:
message = raw_input("Enter a message to encrypt with public key: ")
encrypted_msg = encrypt(public, message)
print "Your encrypted message is: "
print ''.join(map(lambda x: str(x), encrypted_msg))
print "Decrypting message with private key  . . ."

Enter a message to encrypt with public key: hello world!
Your encrypted message is: 
660338289401602022622602103625961684728593566365984071933440338883259826171048120304160298996293925853091751547844670242138781941248603186059839137381492720242401332327685808540980632938377999252416742358526108014738731251334430838443639055254492329868472252284141979410624710289138599176295259582638002040867680959867122431069334839700327665270522819025009558744119274403717441286904442618057221288321837003520637257677627577339523937475960591476120119511103456857651363314401583888222613565583306242010275656219140228418944788209218525000706814842295347667444323632090627331381596145961537658437782315906020453999657525443707994275660670207765208463613806319173580595945057544881019053704343857683850471354268844336268159566745972032612575039585668761555517900884843568935368510180226375081291728336305703352907265878465382389525925138246483430716438137338470054763252389253327061578526587244761344828175316910693

In [17]:
decrypt_start = timeit.default_timer()
decrypted_message =  decrypt(private, encrypted_msg)
decrypt_end = timeit.default_timer()

In [18]:
print "Your original massage is: " + str(decrypted_message)
print("Decryption time is : " + str((decrypt_end - decrypt_start)))

Your original massage is: hello world!
Decryption time is : 0.128057956696


In [19]:
print("setup Time + Decryption Time = " + str((decrypt_end - decrypt_start) + (setup_end - setup_start)))

setup Time + Decryption Time = 1.3090429306
